In [1]:
import networkx as nx
from sentence_transformers import SentenceTransformer
from turingdb_kgsearch.embeddings import build_context_enriched_embeddings, build_sparse_embeddings, build_node2vec_embeddings
from turingdb_kgsearch.search import hybrid_search

# Create knowledge graph
G = nx.DiGraph()
G.add_node("doc1", type="document", text="Machine learning basics")
G.add_node("doc2", type="document", text="Deep learning neural networks")
G.add_node("topic1", type="topic", name="AI")
G.add_edge("topic1", "doc1", rel="contains")
G.add_edge("topic1", "doc2", rel="contains")
print(G)

# Build embeddings
model = SentenceTransformer('paraphrase-MiniLM-L3-v2')
node_vectors_context_heavy, node_texts = build_context_enriched_embeddings(G, model, strategy="heavy")
node_vectors_sparse, _, vectorizer_sparse = build_sparse_embeddings(G, node_texts=node_texts)
node_vectors_node2vec = build_node2vec_embeddings(G, dimensions=384)

# Search
results = hybrid_search(
    query="introduction to neural networks",
    G=G,
    dense_node_vectors=node_vectors_context_heavy,
    sparse_node_vectors=node_vectors_sparse,
    sparse_vectorizer=vectorizer_sparse,
    node_texts=node_texts,  # same node texts used for both dense and sparse
    model=model,
    k=3,
    alpha=0.7,  # 70% semantic, 30% keywords
)

for r in results[:3]:
    print(f"{r['node_id']}: {r['similarity']:.3f}")

DiGraph with 3 nodes and 2 edges
✓ Vector index built using context-enriched embeddings approach (strategy heavy): 3 vectors
Building sparse index (TF-IDF)...
  Using provided node_texts (from dense embeddings)
✓ Sparse index built: 3 vectors
  Vocabulary size: 3
  Sample terms: ['ai', 'topic', 'topic ai']
Training Node2Vec on graph structure...


/home/dev/turingdb-kgsearch/.venv/lib/python3.13/site-packages/node2vec/edges.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/home/dev/turingdb-kgsearch/.venv/lib/python3.13/site-packages/node2vec/edges.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/home/dev/turingdb-kgsearch/.venv/lib/python3.13/site-packages/node2vec/edges.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package o

✓ Node2Vec trained: 3 structural vectors
doc2: 1.000
topic1: 0.593
doc1: 0.300


In [2]:
from turingdb_kgsearch.workflow import search_and_expand_hybrid_filtered

# Stage 1: Find seed nodes via hybrid search
# Stage 2: Traverse graph with semantic+structural filtering
semantic_results, expanded, subgraph = search_and_expand_hybrid_filtered(
    query="risk management AI systems",
    G=G,
    node_vectors=node_vectors_context_heavy,
    node_texts=node_texts,
    sparse_vectors=node_vectors_sparse,
    sparse_vectorizer=vectorizer_sparse,
    structural_vectors=node_vectors_node2vec,
    model=model,
    k_search=3,           # Find 3 seed nodes
    max_hops=2,           # Explore 2 hops in graph
    min_structural_sim=0.7,  # Structural similarity threshold
    min_semantic_sim=0.6,    # Semantic similarity threshold
    structural_weight=0.5,  # 50-50 balance (structure vs. semantic)
    alpha=0.7,  # Weight alpha to attribute to semantic (dense) search, (1 - alpha) for keyword (sparse) search
)

# Subgraph contains relevant nodes + edges + similarity scores
print(f"Found {subgraph.number_of_nodes()} nodes, {subgraph.number_of_edges()} edges")

Stage 1: Hybrid search for 'risk management AI systems'...
--------------------------------------------------------------------------------

Found 3 semantically relevant seed nodes:
  1. topic1 (score: 1.000)
     Topic: AI...
  2. doc2 (score: 0.309)
     doc2. under topic AI...
  3. doc1 (score: 0.300)
     doc1. under topic AI...

Stage 2: Hybrid filtering (structural + semantic)...
  - Max hops: 2
  - Min structural similarity: 0.7
  - Min semantic similarity: 0.6
  - Structural weight: 0.5
--------------------------------------------------------------------------------

  Expanding from: topic1
  Found 0 neighbors (after hybrid filtering):

  Expanding from: doc2
  Found 0 neighbors (after hybrid filtering):

  Expanding from: doc1
  Found 0 neighbors (after hybrid filtering):

Building subgraph...
✓ Subgraph created:
  Total nodes: 3
  - Seed nodes: 3
  - Found neighbors: 0
  - Intermediate nodes: 0
  Total edges: 2
Found 3 nodes, 2 edges


In [3]:
from turingdb_kgsearch.statistics import get_subgraph_stats, print_subgraph_stats

# Comprehensive statistics
stats = get_subgraph_stats(subgraph, include_node_breakdown=True, include_centrality=True, include_paths=True)
print_subgraph_stats(stats, verbose=True)


SUBGRAPH STATISTICS

📊 Basic Metrics:
   nodes: 3
   edges: 2
   density: 0.3333333333333333
   is_connected: True

🔗 Degree Statistics:
   average: 1.33
   max: 2.00
   min: 1.00
   median: 1.00

🏷️  Node Types:
   document: 2
   topic: 1

🎯 Node Roles:
   seed: 3
   found: 0
   intermediate: 0

⭐ Most Central Nodes:
   By Degree:
      topic1: 1.000
      doc1: 0.500
      doc2: 0.500
   By Betweenness:
      doc1: 0.000
      doc2: 0.000
      topic1: 0.000

🔗 Edge Types:
   contains: 2



In [4]:
from turingdb_kgsearch.ranking import rank_nodes_by_importance, print_node_rankings

# Usage examples
rankings = rank_nodes_by_importance(
    subgraph,
    methods="all",  # or ['pagerank', 'degree', 'relevance']
    top_k=10,
    aggregate="average",  # or 'max' or {'pagerank': 0.4, 'degree': 0.3, 'relevance': 0.3}
)

print_node_rankings(rankings, subgraph, show_details=True)


NODE IMPORTANCE RANKINGS

Ranked 3 nodes using: pagerank, degree, betweenness, closeness, eigenvector


📊 PAGERANK (Top 5):
   1. doc1 (document): 0.3701
   2. doc2 (document): 0.3701
   3. topic1 (topic): 0.2597

📊 DEGREE (Top 5):
   1. topic1 (topic): 1.0000
   2. doc1 (document): 0.5000
   3. doc2 (document): 0.5000

📊 BETWEENNESS (Top 5):
   1. doc1 (document): 0.0000
   2. doc2 (document): 0.0000
   3. topic1 (topic): 0.0000

📊 CLOSENESS (Top 5):
   1. doc1 (document): 0.5000
   2. doc2 (document): 0.5000
   3. topic1 (topic): 0.0000

📊 EIGENVECTOR (Top 5):
   1. doc1 (document): 0.7071
   2. doc2 (document): 0.7071
   3. topic1 (topic): 0.0015

⭐ COMBINED RANKING (Top 10):
   1. doc1 (document): 0.6000
   2. doc2 (document): 0.6000
   3. topic1 (topic): 0.2000



In [5]:
from turingdb_kgsearch.visualization import visualize_graph_with_pyvis

# Interactive HTML visualization
visualize_graph_with_pyvis(
    subgraph,
    output_file='graph.html'
)

Auto-generated color map for 2 node types:
  document: #ff6b6b
  topic: #4ecdc4
✓ Interactive graph saved to: graph.html
  Nodes: 3
  Edges: 2


In [6]:
from turingdb_kgsearch.workflow import search_and_expand_hybrid_filtered, generate_report_hybrid_workflow_results

query = "What are the key privacy requirements?"
print(f"Query: '{query}'")

semantic_results, expanded, subgraph = search_and_expand_hybrid_filtered(
    query=query,
    G=G,
    node_vectors=node_vectors_context_heavy,
    node_texts=node_texts,
    sparse_vectors=node_vectors_sparse,
    sparse_vectorizer=vectorizer_sparse,
    structural_vectors=node_vectors_node2vec,
    model=model,
    k_search=5,
    max_hops=10,
    min_structural_sim=0.1,  # Must be structurally similar
    min_semantic_sim=0.1,  # AND semantically relevant to query
    structural_weight=0.5,  # 50-50 balance
    alpha=0.7,  # Weight alpha to attribute to semantic (dense) search, (1 - alpha) for keyword (sparse) search
)

report = generate_report_hybrid_workflow_results(semantic_results, expanded)
print(report)

Query: 'What are the key privacy requirements?'
Stage 1: Hybrid search for 'What are the key privacy requirements?'...
--------------------------------------------------------------------------------

Found 3 semantically relevant seed nodes:
  1. doc2 (score: 1.000)
     doc2. under topic AI...
  2. doc1 (score: 0.653)
     doc1. under topic AI...
  3. topic1 (score: 0.300)
     Topic: AI...

Stage 2: Hybrid filtering (structural + semantic)...
  - Max hops: 10
  - Min structural similarity: 0.1
  - Min semantic similarity: 0.1
  - Structural weight: 0.5
--------------------------------------------------------------------------------

  Expanding from: doc2
  Found 0 neighbors (after hybrid filtering):

  Expanding from: doc1
  Found 0 neighbors (after hybrid filtering):

  Expanding from: topic1
  Found 0 neighbors (after hybrid filtering):

Building subgraph...
✓ Subgraph created:
  Total nodes: 3
  - Seed nodes: 3
  - Found neighbors: -2
  - Intermediate nodes: 0
  Total edges: 2
H

In [7]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

api_keys = {
    "Anthropic": os.getenv("ANTHROPIC_API_KEY"),
    "Mistral": os.getenv("MISTRAL_API_KEY"),
    "OpenAI": os.getenv("OPENAI_API_KEY"),
}

In [8]:
from turingdb_kgsearch.llm import graph_to_llm_context, create_llm_prompt_with_graph, query_llm
from IPython.display import display, Markdown

# Convert graph to LLM-friendly format
graph_text = graph_to_llm_context(subgraph, format='natural')

# Create complete prompt
print(f"Query: '{query}'")
prompt = create_llm_prompt_with_graph(
    query=query,
    subgraph=subgraph,
    report=report,
    format='natural'
)

# Send to your LLM
provider = "Anthropic"

result = query_llm(
    prompt=prompt,
    provider=provider,
    api_key=api_keys[provider],
    temperature=0.2,
)
display(Markdown(result))

Query: 'What are the key privacy requirements?'


Based on the provided graph data, I cannot definitively answer the specific query about "key privacy requirements" because:

1. The graph structure lacks explicit privacy-related content
2. The available nodes (doc1, doc2, topic1) do not show clear privacy details
3. The semantic scores suggest some relevance to AI, but no specific privacy information is visible

Analysis of Available Information:
- The graph contains 3 nodes (2 documents, 1 topic)
- The topic is related to AI
- Documents are connected to the AI topic
- Semantic scores range from 0.300 to 1.000

Recommendation:
To properly answer the privacy requirements query, additional graph context or document content would be needed. The current graph does not provide sufficient information to extract privacy-specific insights.

Limitations of Current Response:
- Insufficient detail about privacy requirements
- No direct privacy-related nodes or edges
- Need more granular document or topic information

Suggested Next Steps:
- Expand graph with more detailed nodes
- Include specific privacy-related content
- Provide fuller document text for semantic analysis